In [1]:
import sys
sys.path.append(r"G:\project\map\wsgi\mindmap\oversea")

In [2]:
import trim_models
reload(trim_models)
from trim_models import *

In [4]:
task_list = CrawlTask.query.all()
urls = []
examples = []
for task in task_list:
    print task.school
    urls.append(task.school_url)
    examples.append(task.example)

Illinois Institute of Technology (IIT Chicago)
Binghamton University(Binghamton)
Colorado School of Mines (CSM Mines)
University of Nevada - Reno(UNR)
University of Central Florida (UCF)
Southern Illinois University Carbondale(SIUC)
University of Kansas (U. Kansas)
Mississippi State University (Missi. SU)
Ball State University (Ball State)
Washington State University (Wshgton SU)
The City College of New York, CUNY (CCNY)
Colorado School of Mines (CSM Mines)
University of Pittsburgh (UPitt)
University of Pittsburgh (UPitt)
Texas Tech University (TTU Whitcre)
Virginia Commonwealth University (VCU)
Cleveland State University (Cleveland)
University of South Florida (USF)
University of Louisiana at Lafayette (U.SW Louisi)
University of South Carolina (U. South Car)
University of Central Florida (UCF)
University of Kansas (U. Kansas)
Iowa State University (Iowa State)
Illinois Institute of Technology (IIT Chicago)
University of Massachusetts Amherst(UMA)
Colorado State University (Col.SU)
Ol

In [29]:
urls = ['http://cs.mines.edu/CS-Faculty',
            'http://science.iit.edu/computer-science/people/faculty',
            'http://cms.bsu.edu/academics/collegesanddepartments/computerscience/facultyandstaff/faculty',
            'http://www.memphis.edu/cs/people/index.php',
            'http://www.smu.edu/Lyle/Departments/CSE/People/Faculty',
            'https://www.csuohio.edu/engineering/eecs/faculty-staff',
            'http://computerscience.engineering.unt.edu/content/faculty',
            'https://www.odu.edu/compsci/research',
            'http://www.cs.ucf.edu/people/index.php',
            'https://www.binghamton.edu/cs/people/faculty-and-staff.html',
            'http://www.lsu.edu/eng/ece/people/index.php',
        'http://www.uwyo.edu/cosc/cosc-directory/',
        'http://www.cs.siu.edu/faculty-staff/continuing_faculty.php',
        'https://www.uml.edu/Sciences/computer-science/faculty/',
        'http://cs.gsu.edu/role/faculty/',
        'http://www.eecs.ku.edu/people/faculty',
        'http://www.cs.iastate.edu/people',
        'http://www.sc.edu/study/colleges_schools/engineering_and_computing/study/areas_of_study/computer_science_and_engineering/our_people/index.php',
        'http://cs.mst.edu/facultystaffandfacilities/facultydirectory/',
        'https://www.lsu.edu/eng/cse/people/faculty/index.php',
        'http://www.cse.msstate.edu/People/Faculty/faculty-staff.php',
        'http://soc.fit.edu/faculty-and-staff.php',
        'http://www.mtu.edu/cs/department/faculty-staff/',
        'https://www.ccny.cuny.edu/people?cat=All&school=21&dept=68',
        'https://www.cise.ufl.edu/people/faculty',
        'http://www.depts.ttu.edu/cs/faculty/',
        'http://www.engineering.pitt.edu/Departments/Electrical-Computer/Faculty/',
        'https://egr.vcu.edu/departments/computer/people/faculty/',
        'http://cs.utdallas.edu/people/faculty/',
        'http://www.usf.edu/engineering/cse/people/index.aspx',
            ]
examples = ['https://inside.mines.edu/CS-Faculty-and-Staff/TracyCamp',
            'http://science.iit.edu/people/faculty/eunice-santos',
            'http://cms.bsu.edu/academics/collegesanddepartments/computerscience/facultyandstaff/faculty/buispaul',
            'http://www.memphis.edu/cs/people/faculty_pages/william-baggett.php',
            'http://www.smu.edu/Lyle/Departments/CSE/People/Faculty/ThorntonMitchell',
            'https://www.csuohio.edu/engineering/charles-k-alexander-professor',
            'http://www.cse.unt.edu/~rakl',
            'https://www.odu.edu/directory/people/a/achernik',
            'http://www.cs.ucf.edu/~bagci',
            'https://www.binghamton.edu/cs/people/kchiu.html',
            'http://www.lsu.edu/eng/ece/people/Faculty/brown.php',
            'http://www.uwyo.edu/cosc/cosc-directory/jlc/index.html',
            'http://www.cs.siu.edu/~abosu',
            'https://www.uml.edu/Sciences/computer-science/faculty/levkowitz-haim.aspx',
            'http://cs.gsu.edu/profile/rafal-angryk/',
            'http://people.ku.edu/~agah/',
            'http://www.cs.iastate.edu/people/pavan-aduri',
            'http://www.sc.edu/study/colleges_schools/engineering_and_computing/faculty-staff/jasonbakos.php',
            'http://cs.mst.edu/facultystaffandfacilities/jiang/',
            'https://www.lsu.edu/eng/cse/people/faculty/karki.php',
            'http://www.cse.msstate.edu/People/Faculty/rikk-anderson.php',
            'http://www.fit.edu/faculty/profiles/profile.php?tracks=wallen',
            'http://www.mtu.edu/cs/department/faculty-staff/faculty/brown/',
            'https://www.ccny.cuny.edu/profiles/michael-anshel',
            'https://www.cise.ufl.edu/people/faculty/lanthony',
            'http://www.depts.ttu.edu/CS/faculty/faculty.php?name=Yong%20Chen',
            'http://www.engineering.pitt.edu/Departments/Electrical-Computer/Faculty/George,-Alan-D-/',
            'http://egr.vcu.edu/directory/tomasz.arodz/',
            'http://cs.utdallas.edu/?p=2507',
            'http://www.usf.edu/engineering/cse/people/andujar-marvin.aspx',
            ]

url_check = {'http://cs.mines.edu/CS-Faculty': 15,
             'http://science.iit.edu/computer-science/people/faculty': 52,
             'http://cms.bsu.edu/academics/collegesanddepartments/computerscience/facultyandstaff/faculty': 19,
             'http://www.memphis.edu/cs/people/index.php': 16,
             'http://www.smu.edu/Lyle/Departments/CSE/People/Faculty': 17,
             'https://www.csuohio.edu/engineering/eecs/faculty-staff': 32,
             'http://computerscience.engineering.unt.edu/content/faculty': 34,
             'https://www.odu.edu/compsci/research': 15,
             'http://www.cs.ucf.edu/people/index.php': 54,
             'https://www.binghamton.edu/cs/people/faculty-and-staff.html': 33,
             'http://www.lsu.edu/eng/ece/people/index.php': 34,
             'http://www.uwyo.edu/cosc/cosc-directory/': 11,
             'http://www.cs.siu.edu/faculty-staff/continuing_faculty.php': 10,
             'https://www.uml.edu/Sciences/computer-science/faculty/': 18,
             'http://cs.gsu.edu/role/faculty/': 31,
             'http://www.eecs.ku.edu/people/faculty': 56,
             'http://www.cs.iastate.edu/people': 51,
             'http://www.sc.edu/study/colleges_schools/engineering_and_computing/study/areas_of_study/computer_science_and_engineering/our_people/index.php': 44,
             'http://cs.mst.edu/facultystaffandfacilities/facultydirectory/': 30,
             'https://www.lsu.edu/eng/cse/people/faculty/index.php': 32,
             'http://www.cse.msstate.edu/People/Faculty/faculty-staff.php': 21,
             'http://soc.fit.edu/faculty-and-staff.php': 14,
             'http://www.mtu.edu/cs/department/faculty-staff/': 27,
             'https://www.ccny.cuny.edu/people?cat=All&school=21&dept=68': 10,
             'https://www.cise.ufl.edu/people/faculty': 1,
             'http://www.depts.ttu.edu/cs/faculty/': 16,
             'http://www.engineering.pitt.edu/Departments/Electrical-Computer/Faculty/': 20,
             'https://egr.vcu.edu/departments/computer/people/faculty/': 5,
             'http://cs.utdallas.edu/people/faculty/': 1,
             'http://www.usf.edu/engineering/cse/people/index.aspx': 10,
             }

In [5]:

import crawler
reload(crawler)


<module 'crawler' from 'G:\project\map\wsgi\mindmap\oversea\crawler.pyc'>

In [6]:
def test_single(i, start = None):
    directory_url = urls[i]
    example = examples[i]
    print i, directory_url #, example
    c = crawler.ResearchCrawler(directory_url, example)
    count, l = c.crawl_faculty_list(directory_url, example)
    # print count, url_check[directory_url]
    # print c.key_words[u'一段研究兴趣的起始词']
    #for e in l:
    #    print e.get("class"), e.parent.get("class")
    #assert url_check[directory_url] <= count, " not beyond"
    faculty_list = []
    
    if start is None:
        end = len(l)
    else:
        end = start + 1
        
    for e in l[start:end]:
        print l.index(e), e.get('href')
        # print e.get_text()
        x = c.dive_into_page(e, True)
        
        #print x['name']
        # print x['tags']
        # print x['position']
        # print x[u'招生意向说明部分原文']
        #return c, e
        #print x['link']
        print x['website']
        # print x['source_name']
        #faculty_list.append(x)
    #return faculty_list, c

crawler.debug_level = " " + " open website"
# faculty_list, c = test_single(12)`
name = 'ualr'
index = 0
for i in range(len(urls)):
    if urls[i].find(name) > 1:
        print i
        t = test_single(i, index)

30
30 http://ualr.edu/computerscience/people/


ReadTimeout: HTTPConnectionPool(host='ualr.edu', port=80): Read timed out. (read timeout=30)

In [42]:
import urlparse

x = "http://www.cs.colostate.edu/~ross/"
y = "http://www.colostate.edu/"
urlparse.urljoin(x, y)

'http://www.colostate.edu/'

In [55]:
import urllib2
href = "Research</span> Interests</b> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;"
href = urllib2.unquote(href.strip())
print href

Research</span> Interests</b> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;


In [ ]:
import re

In [30]:
name = "usf"
index_url = "http://www.usf.edu/"
c = crawler.CollegeCrawler(name, index_url)
crawler.debug_level = " " + " "
l = c.crawl_bfs([index_url])
print len(l)

94
62


In [ ]:
import requests
index_url = "http://ualr.edu/computerscience/people/"
r = requests.get(index_url)
r.raise_for_status()

In [ ]:
import cProfile
crawler = ResearchCrawler()
for url in urls[8:]:
    print url

    result = crawler.crawl_from_directory(url, examples[urls.index(url)])
    print len(result)
    print result[0]

In [ ]:

import pstats  
p = pstats.Stats('prof.log')

In [ ]:
p.sort_stats('cumtime')
p.print_stats()